# TP traitement de données en tables

Python possède une bibliothèque standard pour le traitement des fichiers csv.
Voici la [documentation de la bibliothèqe csv](https://docs.python.org/3/library/csv.html).


Dans ce TP nous allons travailler sur des données des collèges du Finistère qui sont disponibles depuis une URL.

Dans une première étape nous allons voir comment automatiser la récupération de ces données et leur transformation en une liste de dictionnaires Python.

In [ ]:
# importation des bibliothèques nécessaires
import requests
import os
import csv

def recupere_donnes():
    """
    Retourne les données sous forme d'une liste de dictionnaires
    """
    local_file = "college_finistere.csv" #nom donné au fichier
    # recuperation des données depuis l'url
    url = "https://geobretagne.fr/geoserver/cd29/wfs?SERVICE=WFS&REQUEST=GetFeature&VERSION=2.0.0&TYPENAMES=cd29%3Acolleges_29&OUTPUTFORMAT=csv"
    data = requests.get(url).content # on récupère les données sous forme de bytes
    # création d'un fichier et écriture des données
    with open(local_file, 'wb') as csvfile: # 'w' pour write et 'b' pour bytes
        csvfile.write(data)
    
    donnes = []
    # ouverture du fichier précédemment créé
    with open(local_file) as csvfile:
        reader = csv.reader(csvfile)
        for ligne in reader:
            donnes.append(ligne)
    return donnes
        
etablissements = recupere_donnes()
print(etablissements[0:10])

**Remarques**

- Le fichier que l'on ouvre est en mode **bytes** car nous recevons les données sous forme d'un flux d'octets et non de caractères. Pour s'en convaincre, faire un print(data)
- On utilise l'instruction *with* qui garantit que si une erreur se produit dans le with, le fichier sera correctement fermé. [voir l'instruction with](https://docs.python.org/fr/3/reference/compound_stmts.html#the-with-statement)


Répondre au questions suivantes:

    1. Quel est le type de chaque élément de la liste retournée?
    2. Combien d'éléments possède chaque élement de la liste?
    3. Quel est le délimiteur des données dans l'url utilisée? 
    4. Quels sont les colonnes dont il faudra changer le type?

**Travail à faire:**

Modifier la fonction *recupere_donnes* afin que:

- Chaque ligne ait bien le bon nombre d'éléments;
- Chaque ligne soit un dictionnaire (voir la documentation de la bibliothèque);
- Les colonnes aient le bon type;
- On ne télécharge les données de l'URL uniquement si le fichier local n'est pas présent.

## Filtre des données

Ecrire une fonction *filtre(donnees, criteres)* qui prend en paramètre:
- *donnes* : une liste de dictionnaires
- *criteres* : un dictionnaire dont les clés sont les noms des colonnes et les valeurs sont les valeurs recherchées

et qui retourne une liste filtrée de dictionnaires dont les critères correspondent aux valeurs recherchées (on utilisera un ET entre les critères)

In [ ]:
def filtrer(donnes, criteres):
    pass

Avec ce filtre:

- Retourner tous les établissements de Chateaulin;
- Retourner tous les établissements publics de Brest;
- Retourner tous les établissements qui ont comme code postal 29000 et sont privés.

**Bonus**:
Utiliser une liste par compréhension pour réaliser votre filtre.

**A faire**:

Ajouter à la fonction *filtrer*  un paramètre nommé *colonnes*, qui par défaut vaut *None*.
Si *colonnes* est une liste de noms de colonnes alors seules ces colonnes seront retournées.

<div id="tri"></div>

## Tri des données

Avant de continuer, assurez vous que le cours sur le **[tri](3_tri.ipynb)** des données a été lu.

Dans la partie précédente de ce TP nous avons vu comment **filtrer** nos données. Nous allons maintenant découvrir comment **trier** nos données.

**A faire**:

En utilisant la fonction *sorted* de Python, écrire une fonction *trier* qui prend en paramètres :
- *donnees* : une liste de dictionnaires
- *criteres*, une liste de string représentant les tris de colonnes à effectuer.


In [1]:
def trier(donnees, criteres):
    """
    donnees (liste) : une liste de colleges
    critères (liste) : la liste des colonnes à trier
    Retourne la liste triée des colleges.
    """
    pass

En utilisant les fonctions que vous avez codées :

- Afficher les collèges de Brest, triés par statut puis par nom, en affichant le nom, le statut et l'adresse.

<div id="fusion"></div>

## Fusion de données

Avant de continuer, assurez vous que le cours sur la **[fusion](4_fusion.ipynb)** des données a été lu.

Pour fusionner plusieurs tables nous allons utiliser la librairie [pandas](https://pandas.pydata.org/docs/index.html).

La liste des collèges du Morbihan est disponible sur le web, voici les données.

In [ ]:
import pandas as pd
url = "https://www.data.gouv.fr/fr/datasets/r/bf9d46b1-5430-4866-ab7d-58a4d794324d"
colleges_morbihan = pd.read_csv(url, delimiter=';', encoding='iso 8859-15')
colleges_morbihan

De nombreuses données sont disponibles sur l'ensemble des collèges de la France. (Attendre, les données sont volumineuses)

In [ ]:
import os
import pandas as pd
import requests

# le fichier est volumineux, à télécharger une seule fois
local_file = "geolocalisation.csv"
if not local_file in os.listdir():
    url = "https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B"
    data = requests.get(url).content
    with open(local_file, 'wb') as csvfile:
        csvfile.write(data)
geolocalisations = pd.read_csv(local_file, delimiter = ";")
geolocalisations

Il y a 35 colonnes dans les données !!!

Seuls, Code établissement, longitude et la latitude nous intéresse.

In [ ]:
geolocalisations = geolocalisations.filter(axis='columns', items=['Code établissement', 'Latitude', 'Longitude'])
geolocalisations

**Problème**
- Le code établissement ne possède pas la même dénomination dans les 2 tables.

**solution**
- Il est possible de faire **correspondre** le nom des colonnes communes avec un *mapper*.


In [ ]:
mapper = {'CODE':'Code établissement',
         }

colleges_morbihan.rename(columns=mapper, inplace=True)
print("La liste des établissements du morbihan")
colleges_morbihan

**A faire**:

En vous inspirant du cours sur la **[fusion](4_fusion.ipynb)**, trouver une solution pour ajouter la localisation (longitude, latitude) dans la table des collèges du Morbihan.

**A Faire**

Utiliser [la bibliothèque Folium](https://python-visualization.github.io/folium/quickstart.html#Getting-Started) pour afficher les collèges du Morbihan sur une carte.

Indices:
- Pour centrer la carte, écrire "en dur" une localisation d'un collège.
- Utiliser un zoom_start de 9.
- Renseigner vous sur comment [itérer sur les lignes d'un tableau pandas](https://www.google.com/search?q=iterate+rows+on+panda+dataframe&ei=x9_7Y9CpIoW7kdUP_8ihgAI&ved=0ahUKEwiQqevaoLT9AhWFXaQEHX9kCCAQ4dUDCA8&uact=5&oq=iterate+rows+on+panda+dataframe&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQAzILCAAQHhANEPEEEAo6CggAEEcQ1gQQsAM6BQgAEKIEOg0IABAIEB4QDRDxBBAKOgsIABAIEB4QDRDxBEoECEEYAEoFCEASATFQowhYoA9ggBFoAXABeACAAZQBiAHSBJIBAzAuNZgBAKABAcgBCMABAQ&sclient=gws-wiz-serp).

In [ ]:
import folium
